In [75]:
import math

class Value:

    def __init__(self, data, _children = (), _op = ''):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.grad = 0.0
        self._backward = lambda: None

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad = 1.0*out.grad
            other.grad = 1.0*out.grad

        out._backward = _backward

        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad = out.grad*other.data
            other.grad = out.grad*self.data

        out._backward = _backward

        return out

    def tanh(self):
        n = self.data
        t = (math.exp(2*n)-1)/(math.exp(2*n)+1)
        out = Value(t, (self,), 'tanh')

        def _backward():
            self.grad = (1 - t**2)*out.grad

        out._backward = _backward
        return out
    
    def backward(self):

        self.grad = 1.0

        topo = []
        visited = set()


        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)

        for node in reversed(topo):
            node._backward()
        


In [76]:
a = Value(2.0)
b = Value(3.0)
c = Value(4.0)

e = a*b
d = e + c
f = Value(-2.0)
L = d + f
L

Value(data=8.0)

In [81]:
L.backward()
print(a.grad)

3.0


# Topological sort to back-propagate

In [50]:
## code moved to Value class above

In [68]:
print(e.grad)

0.0
